In [ ]:
!pip install autots
from autots import AutoTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.7/713.7 kB 6.3 MB/s eta 0:00:00


In [ ]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd

# Define the list of FAANG ticker symbols
tickers = ['AAPL', 'AMZN', 'NFLX', 'GOOGL']

# Set the start and end dates for the historical data
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=30*365)).strftime('%Y-%m-%d')

# Create an empty dataframe to store the stock data
combined_data_open = pd.DataFrame()
combined_data_close = pd.DataFrame()
combined_data_high = pd.DataFrame()
combined_data_low = pd.DataFrame()
combined_data_vol = pd.DataFrame()
combined_data_adj_cls = pd.DataFrame()
combined_data = pd.DataFrame()

# Download historical stock data for each ticker and combine into a single dataframe
for ticker in tickers:
    print(ticker)
    data = yf.download(ticker, start=start_date, end=end_date)
    data.columns = ticker+"_"+data.columns  # Add a column to identify the ticker
    if combined_data_open.shape[0] ==0:
      combined_data = data
      combined_data_adj_cls = data[ticker+"_Adj Close"]
      combined_data_open = data[ticker+"_Open"]
      combined_data_close = data[ticker+"_Close"]
      combined_data_high = data[ticker+"_High"]
      combined_data_low = data[ticker+"_Low"]
      combined_data_vol = data[ticker+"_Volume"]
    else:
      combined_data = pd.merge(combined_data, data, left_index=True, right_index=True)
      combined_data_open = pd.merge(combined_data_open, data[ticker+"_Open"], left_index=True, right_index=True)
      combined_data_adj_cls = pd.merge(combined_data_adj_cls, data[ticker+"_Adj Close"], left_index=True, right_index=True)
      combined_data_close = pd.merge(combined_data_close, data[ticker+"_Close"], left_index=True, right_index=True)
      combined_data_high = pd.merge(combined_data_high, data[ticker+"_High"], left_index=True, right_index=True)
      combined_data_low = pd.merge(combined_data_low, data[ticker+"_Low"], left_index=True, right_index=True)
      combined_data_vol = pd.merge(combined_data_vol, data[ticker+"_Volume"], left_index=True, right_index=True)



AAPL
[*********************100%***********************]  1 of 1 completed
AMZN
[*********************100%***********************]  1 of 1 completed
NFLX
[*********************100%***********************]  1 of 1 completed
GOOGL
[*********************100%***********************]  1 of 1 completed


In [ ]:
def preprocess(data):
  df2 = data.copy()
  num_lags = 10
  delay = 1 # predict target one step ahead
  for column in df2:
      for lag in range(1,num_lags+1):
          df2[column + '_lag' + str(lag)] = df2[column].shift(lag*-1-(delay-1))
          if column != 'wnd_dir':
              df2[column + '_avg_window_length' + str(lag+1)] = df2[column].shift(-1-(delay-1)).rolling(window=lag+1,center=False).mean().shift(1-(lag+1))

  df2.dropna(inplace=True)

  mask = (df2.columns.str.contains('GOOGL_Adj Close') | df2.columns.str.contains('lag') | df2.columns.str.contains('window'))
  df_processed = df2[df2.columns[mask]]
  return df_processed

# the columns in the processed dataframe
process = False

if process == True:
  df_processed = preprocess(combined_data_adj_cls)
else:
  df_processed = combined_data_adj_cls

df_processed.columns

Index(['AAPL_Adj Close', 'AMZN_Adj Close', 'NFLX_Adj Close',
       'GOOGL_Adj Close'],
      dtype='object')

In [ ]:
df_processed.head(10)

,AAPL_Adj Close,AMZN_Adj Close,NFLX_Adj Close,GOOGL_Adj Close
Date,,,,
2004-08-19,0.466103,1.9315,2.264286,2.511011
2004-08-20,0.467469,1.9755,2.262857,2.710460
2004-08-23,0.471718,1.9725,2.272857,2.737738
2004-08-24,0.484923,1.9525,2.208571,2.624374
2004-08-25,0.501618,2.0150,2.238571,2.652653
2004-08-26,0.526054,2.0095,2.177143,2.700450
2004-08-27,0.521349,1.9950,2.164286,2.656406
2004-08-30,0.517858,1.9155,2.064286,2.552803
2004-08-31,0.523474,1.9070,1.992857,2.561812


In [ ]:
df_processed['ds'] = df_processed.index

In [ ]:
df_processed.shape

(4765, 5)

In [ ]:
df_processed.rename(columns = {'GOOGL_Adj Close':'y'}, inplace = True)
df_processed.head()
df_processed['ds']= pd.to_datetime(df_processed['ds'])

In [ ]:
# df_processed.reset_index(inplace = True, drop = True)

In [ ]:
df_processed.shape

(4765, 5)

In [ ]:
df_train = df_processed[:int(0.8*df_processed.shape[0])]
df_test = df_processed[int(0.8*df_processed.shape[0]):]

In [ ]:
df_test.shape

(953, 5)

In [ ]:
df_train

,AAPL_Adj Close,AMZN_Adj Close,NFLX_Adj Close,y,ds
Date,,,,,
2004-08-19,0.466103,1.931500,2.264286,2.511011,2004-08-19
2004-08-20,0.467469,1.975500,2.262857,2.710460,2004-08-20
2004-08-23,0.471718,1.972500,2.272857,2.737738,2004-08-23
2004-08-24,0.484923,1.952500,2.208571,2.624374,2004-08-24
2004-08-25,0.501618,2.015000,2.238571,2.652653,2004-08-25
...,...,...,...,...,...
2019-10-03,53.764645,86.221001,268.149994,59.471500,2019-10-03
2019-10-04,55.271759,86.982498,272.790009,60.548000,2019-10-04
2019-10-07,55.283939,86.633003,274.459991,60.412498,2019-10-07


In [ ]:
model = AutoTS(
    forecast_length=953,
    prediction_interval=0.95,
    model_list = ['VECM',
 'DynamicFactor',
 'GluonTS',
 'RollingRegression',
 'WindowRegression',
 'VAR',
 'MultivariateMotif',
 'NVAR'],#]'multivariate',# or ['ARIMA','ETS']
    max_generations=1,
    num_validations=1,
    no_negatives=True,
    n_jobs='auto')

model.fit(df_train)



Using 1 cpus for n_jobs.
Inferred frequency is: B
Model Number: 1 with model GluonTS in generation 0 of 1
Template Eval Error: ImportError('GluonTS installation is incompatible with AutoTS. The numpy version is sometimes the issue, try 1.23.1 {as of 06-2023}') in model 1 in generation 0: GluonTS
Model Number: 2 with model GluonTS in generation 0 of 1
Template Eval Error: ImportError('GluonTS installation is incompatible with AutoTS. The numpy version is sometimes the issue, try 1.23.1 {as of 06-2023}') in model 2 in generation 0: GluonTS
Model Number: 3 with model GluonTS in generation 0 of 1
Template Eval Error: ImportError('GluonTS installation is incompatible with AutoTS. The numpy version is sometimes the issue, try 1.23.1 {as of 06-2023}') in model 3 in generation 0: GluonTS
Model Number: 4 with model GluonTS in generation 0 of 1
Template Eval Error: ImportError('GluonTS installation is incompatible with AutoTS. The numpy version is sometimes the issue, try 1.23.1 {as of 06-2023}'

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but ExtraTreesRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but ExtraTreesRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but ExtraTreesRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but ExtraTreesRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but ExtraTreesRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have 

Model Number: 19 with model VECM in generation 0 of 1
Template Eval Error: ValueError("regression_type='User' but no future_regressor supplied") in model 19 in generation 0: VECM
Model Number: 20 with model DynamicFactor in generation 0 of 1
Template Eval Error: ZeroDivisionError('integer division or modulo by zero') in model 20 in generation 0: DynamicFactor
Model Number: 21 with model GluonTS in generation 0 of 1
Template Eval Error: ImportError('GluonTS installation is incompatible with AutoTS. The numpy version is sometimes the issue, try 1.23.1 {as of 06-2023}') in model 21 in generation 0: GluonTS
Model Number: 22 with model RollingRegression in generation 0 of 1
Template Eval Error: ValueError('future_regressor not supplied, necessary for regression_type') in model 22 in generation 0: RollingRegression
Model Number: 23 with model WindowRegression in generation 0 of 1
Model Number: 24 with model VAR in generation 0 of 1
Model Number: 25 with model MultivariateMotif in generation 

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:2418: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.277738459730434e+39, tolerance: 6.903566029158706e+35
  ) = cd_fast.enet_coordinate_descent_multi_task(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MultiTaskElasticNet was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MultiTaskElasticNet was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MultiTaskElasticNet was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MultiTaskElasticNet was fitte

Model Number: 33 with model DynamicFactor in generation 0 of 1


...

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


12 - DynamicFactor with avg smape 41.02: 
Model Number: 13 of 15 with model VECM for Validation 1
📈 13 - VECM with avg smape 33.54: 
Model Number: 14 of 15 with model VAR for Validation 1
14 - VAR with avg smape 53.66: 
Model Number: 15 of 15 with model DynamicFactor for Validation 1


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


15 - DynamicFactor with avg smape 49.77: 


Initiated AutoTS object with best model: 
MultivariateMotif
{'fillna': 'zero', 'transformations': {'0': 'Detrend', '1': 'bkfilter', '2': 'PowerTransformer', '3': 'AlignLastValue'}, 'transformation_params': {'0': {'model': 'GLS', 'phi': 1, 'window': None}, '1': {}, '2': {}, '3': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 1.0, 'first_value_only': False}}}
{'window': 28, 'point_method': 'mean', 'distance_metric': 'correlation', 'k': 10, 'max_windows': 1000}
Validation: 0, 1
SMAPE: 26.1989165999426, 37.281059719956495
MAE: 9.375188225130243e+16, 8.154436625349336e+16
SPL: 8.78425499704309, 20.049534130103957

In [ ]:
prediction = model.predict()

In [ ]:
forecast = prediction.forecast

In [ ]:
len(forecast)

In [ ]:
prediction

In [ ]:
y_act = df_processed['y']
y_act.reset_index(inplace = True, drop=True)
y_act

In [ ]:
y_pred = predictions['yhat']
y_pred.reset_index(inplace = True, drop=True)
y_pred

In [ ]:
from sklearn.metrics import mean_squared_error
import math
rmse = round(math.sqrt(mean_squared_error(predictions['yhat'], y_act)),2)


In [ ]:
plt.plot(y_act, label = 'Actual')
plt.plot(predictions['yhat'], label = "Predicted")
plt.legend()

plt.title("AutoTS output RMSE - "+ str(rmse) )